In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import config
import glob
from tqdm import tqdm

In [3]:
import cv2
import geopandas as gpd
from skimage import io, img_as_ubyte
import rasterio as rs
from rasterio.plot import show
from rasterio import features as feat

ModuleNotFoundError: No module named 'cv2'

In [ ]:
project_root = config.project_root
sn6_dataset = os.path.join(project_root, 'dataset', 'data-spacenet6')
train_dir = os.path.join(sn6_dataset, "train", "AOI_11_Rotterdam")
test_dir = os.path.join(sn6_dataset, "test_public", "AOI_11_Rotterdam")
mask_train_dir =  os.path.join(train_dir, "masks")

In [ ]:
def get_test_path(image_id):
    return f'{test_dir}/SAR-Intensity/SN6_Test_Public_AOI_11_Rotterdam_{image_id}.tif'

In [ ]:
def get_test_image_ids(test_dir):
    search_pattern = os.path.join(test_dir, 'SAR-Intensity', 'SN6_Test_Public_AOI_11_Rotterdam_*.tif')
    file_paths = glob.glob(search_pattern)
    image_ids = [os.path.basename(file_path).replace('SN6_Test_Public_AOI_11_Rotterdam_', '').replace('.tif', '') for file_path in file_paths]
    return image_ids


In [ ]:
test_image_ids = get_test_image_ids(test_dir)
print(len(test_image_ids))

In [ ]:
if not os.path.exists(mask_train_dir):
    os.mkdir(mask_train_dir)

In [ ]:
summary_data = os.path.join(train_dir, 'SummaryData', 'SN6_Train_AOI_11_Rotterdam_Buildings.csv')
print(summary_data)
summary_df = pd.read_csv(summary_data)

In [ ]:
image_ids = summary_df.ImageId.unique()
len(image_ids)

In [ ]:
summary_df.head()

In [ ]:
def get_geojson(image_id):
    return f"{train_dir}/geojson_buildings/SN6_Train_AOI_11_Rotterdam_Buildings_{image_id}.geojson"  

def get_filepath(image_id, mode):
    return f'{train_dir}/{mode}/SN6_Train_AOI_11_Rotterdam_{mode}_{image_id}.tif'

def get_raster(image_id, mode):
    return rs.open(get_filepath(image_id, mode))

def get_mask_filepath(image_id,  mode='SAR-Intensity'):
    return f'{mask_train_dir}/SN6_Train_AOI_11_Rotterdam_{mode}_{image_id}.png'

In [ ]:
show(get_raster(image_ids[0], mode='SAR-Intensity'))

In [ ]:
df = gpd.read_file(get_geojson(image_ids[0]))

In [ ]:
def get_binary_mask(image_id, raster):
    gdf = gpd.read_file(get_geojson(image_id))
    if gdf.shape[0] == 0:
        mask = np.zeros((raster.height, raster.width))
    
    mask = feat.geometry_mask(
        gdf.geometry,
        out_shape=(raster.height, raster.width),
        transform=raster.transform,
        invert=True,
    )
    return mask

In [ ]:
import matplotlib.pyplot as plt

mask = get_binary_mask(image_ids[0], get_raster(image_ids[0], mode='SAR-Intensity'))

plt.imshow(mask, cmap='gray')
plt.show()

In [ ]:
# not_processed = []
# for image_id in tqdm(image_ids):
#     try:
#         mode = 'SAR-Intensity'
#         raster = get_raster(image_id, mode)
#         mask = get_binary_mask(image_id, raster)
#         mask_ubyte = img_as_ubyte(mask)  # Convert the boolean mask to an 8-bit unsigned integer mask
#         out_filename = f'SN6_Train_AOI_11_Rotterdam_{mode}_{image_id}.png'
#         io.imsave(os.path.join(mask_train_dir, out_filename), mask_ubyte)
#     except Exception as e:
#         print(f"Error processing image {image_id}: {e}")
#         not_processed.append(image_id)

# print(f"{len(not_processed)} images could not have binary masks saved/generated.")

The error:
Error processing image 20190823142111_20190823142355_tile_3948: No valid geometry objects found for rasterize
Error processing image 20190823142111_20190823142355_tile_3943: No valid geometry objects found for rasterize
Error processing image 20190823125228_20190823125530_tile_8582: No valid geometry objects found for rasterize


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rs
from rasterio.plot import show
from skimage import io

In [ ]:
def visualize(image_id):
    sar_image = rs.open(get_filepath(image_id, mode='SAR-Intensity'))
    mask = io.imread(get_mask_filepath(image_id))

    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 10))


    show(sar_image, ax=ax1, cmap='gray')
    ax1.set_title('SAR Image')
    ax1.axis('off')


    show(sar_image, ax=ax2, cmap='gray')
    ax2.imshow(mask, alpha=0.5)
    ax2.set_title('Mask')
    ax2.axis('off')

    plt.show()

In [ ]:
visualize(image_ids[11])

In [ ]:
def plot_prediction():
    pass

In [ ]:
import rasterio

def inspect_image(image_path):
    with rasterio.open(image_path) as src:
        print("Image shape:", src.shape)
        print("Number of channels:", src.count)
        print("Image CRS:", src.crs)
        print("Image transform:", src.transform)

image_path = 'path/to/your/SAR/intensity/image.tif'
inspect_image(image_path)
